In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'

# from __future__ import absolute_import, division, print_function
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error as mse
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential,Model
from keras.layers import Input,LSTM, Dense, Flatten, Conv1D, Lambda, Reshape
from keras.layers.merge import concatenate, multiply,add
import tensorflow as tf
from keras import regularizers
from keras.initializers import glorot_uniform
from tqdm import tqdm
from keras import regularizers
from statsmodels.tsa.seasonal import seasonal_decompose
import datetime



import rpy2
import rpy2.robjects.numpy2ri
from stldecompose import decompose
rpy2.robjects.numpy2ri.activate()
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()
stats = importr('stats')
stl=stats.stl
ts =stats.ts

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data= pd.read_csv("Data/FromMohammed/cif_dataset_complete.csv",header=None)
predictions = pd.read_csv("Data/FromMohammed/theta_25_h3.csv",index_col=0,skiprows = [1])

In [3]:
print(data.shape)
data.head(5)

(72, 123)


,0,1,2,3,4,5,6,7,8,9,...,113,114,115,116,117,118,119,120,121,122
0,ts1,12,monthly,855.144866,847.292062,824.731660,866.645406,928.333300,927.649280,901.072255,...,1585.222987,1674.195284,1674.955086,1695.759843,1655.823053,1672.327141,1650.605735,1738.656214,1710.378030,1683.418022
1,ts2,12,monthly,496.999629,491.266202,514.497083,505.818719,523.568994,525.329842,529.642170,...,720.558589,743.722838,726.890762,739.548510,744.333120,740.740383,752.161177,756.418571,758.655439,768.512031
2,ts3,12,monthly,1333.548021,1370.089237,1415.076738,1281.416912,1358.059752,1384.964713,1276.752289,...,792.898046,732.690151,773.346272,782.951732,749.658718,784.576004,786.650821,672.321333,637.252308,725.853345
3,ts4,12,monthly,601.167280,541.750384,601.225895,561.288487,543.068962,560.192329,585.670783,...,1019.850913,993.553967,1000.544495,981.014694,989.564612,1044.511185,1095.874066,1117.322564,1106.108627,1066.770728
4,ts5,12,monthly,507.766151,498.915098,513.571657,507.190113,510.092496,506.453491,514.185068,...,577.076351,566.641173,604.329118,580.958679,637.388618,635.713882,634.583760,691.642323,710.045035,739.101974


In [4]:
print(predictions.shape)
predictions.head(5)

(72, 90)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90
2,1081.444132,1087.148239,1092.882433,1100.117805,1105.274186,1110.454735,1110.426923,1114.837906,1119.240180,1126.874229,...,1620.843678,1626.965266,1631.962689,1636.952468,1664.145022,1669.711184,1675.295964,1674.559189,1679.626827,1684.686831
3,561.950940,563.277718,564.602934,561.381595,562.593660,563.805725,569.279245,570.484922,571.693153,576.019832,...,732.844422,729.778800,731.048288,732.316675,735.430566,736.659927,737.888263,743.791793,745.192655,746.596156
4,1179.646121,1175.349049,1171.051976,1164.856043,1160.804603,1156.753162,1135.928298,1131.794602,1127.660906,1153.695571,...,770.776280,778.943087,776.132909,773.338200,764.614275,761.654324,758.711887,764.080298,761.596104,759.124258
5,592.584192,593.290366,593.997825,595.661561,596.470380,597.280836,594.136184,594.599537,595.063449,597.309465,...,959.384016,1004.755948,1011.739995,1018.799201,1006.294718,1014.192403,1022.182148,1084.620448,1091.219892,1097.819337
6,511.011338,511.094677,511.178030,515.664772,515.873136,516.081669,513.953060,514.042724,514.132403,518.869072,...,525.345368,568.710323,569.531192,570.353247,586.182927,587.642310,589.107559,637.161309,640.018150,642.887800


In [5]:
def make_input(data,window_size,horizon=1):
    length=data.shape[0]
#     depth=data.shape[2]
    y = np.zeros([length-window_size+1-horizon,horizon])
    output=np.zeros([length-window_size+1-horizon,window_size])
    for i in range(length-window_size+1-horizon):
        output[i:i+1,:]=data[i:i+window_size]
        y[i,:]= data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],window_size,1), y
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def make_k_input(data,window_size,horizon):
    length = data.shape[0]
    output= np.zeros([length-window_size+1-horizon,horizon])
    for i in range(length-window_size-horizon+1):
        output[i:i+1,:]=data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],horizon,1)
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def nonov_make_input(data,window_size,horizon=1):
    length=data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon
#     print(str(extra))
    data = np.append(data,np.zeros([horizon-extra]))
#     print(data)
    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
        
    output=np.zeros([i_val,window_size])
    y=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[i*horizon:(i*horizon)+window_size]
        y[i,:]= data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
        
    return output.reshape(output.shape[0],window_size,1), y
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def nonov_make_k_input(data,window_size,horizon):
    length = data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon
    data_app = np.repeat(data[-1],extra)
    data = np.append(data,data_app)    
#     data = np.append(data,np.zeros([horizon-extra]))
    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
    output=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
    return output.reshape(output.shape[0],horizon,1)    

In [6]:
def sMAPE(val,preds,horizon):
    temp = np.abs(val-preds)
    temp1=np.abs(val)+np.abs(preds)    
    smape = 2/horizon*np.sum(temp/temp1)
    return smape

def sMAPE1(val,preds):
    temp = np.abs(val-preds)
    temp1=np.abs(val)+np.abs(preds)    
    smape = 2/np.sum(temp/temp1)
    return smape

# Case 0:
##### Using only data (no prediction)
##### Using Decomposition Method
##### Using 3Layers Conv (64) Neural Network 

In [ ]:
print(datetime.datetime.now())
smape_pred=0
smape1_pred=0
smape_theta=0
smape1_theta=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    for horizon in [1]:
        final_predictions = np.zeros([data_length,12])
        for y in range(data_length):
############################################################################################
######################creating train, validation and test sets from Data####################
############################################################################################
            horizon = data.iloc[y].values[1]
            window_size=7
            current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
            #rr = current_row.size
            #rr = int(np.floor(rr*.25))
            #temp1=current_row[rr:] # values after (rr)th position in current_row
            temp1=current_row
            epsilon = 0.05     #check
            temp1[temp1<epsilon] = temp1[temp1<epsilon]+0.05
            temp1 = np.log(temp1)

####################################  Decomposition ##########################
            frequency = 7     #check
            result = stl(ts(temp1,frequency=frequency),"periodic")
            result=pandas2ri.ri2py(result.rx2('time.series')) # result[:,0]->seasonal // result[:,1]->trend //  result[:,2]->residual
            series  = result[:,0]+result[:,2]
            #series_d  = result[:,0]+result[:,1]+result[:,2]#check
#############################################################################
            #series=temp1
            series_data = series_d[:-horizon]
            series_length = series_data.size
            n_val = int(np.round(series_length*.2))
            
            train = series_data[:-n_val]
            test = series_d[-(horizon+window_size):]
            #val = series_data[-(n_val+window_size+3):]
            val = series_data[-(2*n_val+window_size):]

            train_sequence = make_input(train, window_size,horizon)
            val_sequence = make_input(val,window_size,horizon)
            test_sequence = nonov_make_input(test,window_size,horizon)

            x_train = train_sequence[0]
            y_train =train_sequence[1]
            x_val = val_sequence[0]
            y_val = val_sequence[1]
            x_test = test_sequence[0]
            y_test = test_sequence[1]

############################################################################################
###############creating train, validation and test sets from Predictions####################
############################################################################################
            #current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
            #current_pred[np.argwhere(current_pred<=0)]=0.5
            #current_pred = np.log(current_pred)
####################################  Decomposition ##########################
            #result_pred = stl(ts(current_pred,frequency=frequency),"periodic")
            #result_pred=pandas2ri.ri2py(result_pred.rx2('time.series'))
            ##series_pred = result_pred[:,0]+result_pred[:,2]
            #series_p = result_pred[:,0]+result_pred[:,1]+result_pred[:,2]#check
#############################################################################
            ##series_pred=current_pred
            #series_pred=series_p[:-horizon]
            #train_p = series_pred[:-n_val]
            ##val_p = series_pred[-(n_val+window_size+3):]
            #val_p = series_pred[-(2*n_val+window_size):]
            #test_p = series_p[-(horizon+window_size):]

            #train_pred = make_k_input(train_p,window_size,horizon)
            #val_pred = make_k_input(val_p,window_size,horizon)
            #test_pred = nonov_make_k_input(test_p,window_size,horizon)


            ##train_pred=train_pred.reshape(train_pred.shape[0],horizon)
            ##val_pred=val_pred.reshape(val_pred.shape[0],horizon)
            ##test_pred=test_pred.reshape(test_pred.shape[0],horizon)
############################################################################################
########################    Creating inputs for Neural Network      ########################
############################################################################################
            #train_input = np.append(x_train,train_pred,axis=1)
            train_input = x_train
            #val_input = np.append(x_val,val_pred,axis=1)
            val_input = x_val
            #test_input = np.append(x_test,test_pred,axis=1)
            test_input = x_test

############################################################################################
########################       Neural Network Configuration         ########################
############################################################################################
            #input_data= Input(batch_shape=(None,window_size+horizon,1),name='input_data')
            input_data= Input(batch_shape=(None,window_size,1),name='input_data')

            branch_0 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
            branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
            branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
            branch_3=Flatten()(branch_2)
            net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

            model=Model(inputs=[input_data],outputs=net)
            callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF0.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

            model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)
            model.load_weights('MyModelCheckpoint/CIF0.h5')
            pred=model.predict({'input_data':test_input})
            pred=pred.reshape(pred.size)[:horizon]

            final_predictions[y,:horizon] = pred.reshape(horizon)
            smape1_pred+=sMAPE1(pred,test[-horizon:])
            smape_pred+=sMAPE(pred,test[-horizon:],horizon)
            smape1_theta+=sMAPE1(np.exp(current_pred[-horizon:]),test[-horizon:])
            smape_theta+=sMAPE(np.exp(current_pred[-horizon:]),test[-horizon:],horizon)
            pbar.update(1)
        np.savetxt('MyModelCheckpoint/prediction0.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(smape1_pred)
print('-----------------')
print(smape_pred)
print('-----------------')
print(smape1_theta)
print('-----------------')
print(smape_theta)
print(datetime.datetime.now())

2019-07-20 19:39:49.593786


100%|████████████████████████████████████████████████████████████████████████████████| 72/72 [1:19:48<00:00, 77.69s/it]


28.990386264011082
-----------------
77.03452912581464
-----------------
14.630443377960347
-----------------
142.71364646030867
2019-07-20 20:59:37.699052


# Case 1:
##### Using only data (no prediction)
##### Without Decomposition 
##### Using 3Layers Conv (64) Neural Network 

In [8]:
print(datetime.datetime.now())
smape_pred=0
smape1_pred=0
smape_theta=0
smape1_theta=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    for horizon in [1]:
        final_predictions = np.zeros([data_length,12])
        for y in range(data_length):
############################################################################################
######################creating train, validation and test sets from Data####################
############################################################################################
            horizon = data.iloc[y].values[1]
            window_size=7
            current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
            #rr = current_row.size
            #rr = int(np.floor(rr*.25))
            #temp1=current_row[rr:] # values after (rr)th position in current_row
            temp1=current_row
            epsilon = 0.05     #check
            temp1[temp1<epsilon] = temp1[temp1<epsilon]+0.05
            temp1 = np.log(temp1)

####################################  Decomposition ##########################
            #frequency = 7     #check
            #result = stl(ts(temp1,frequency=frequency),"periodic")
            #result=pandas2ri.ri2py(result.rx2('time.series')) # result[:,0]->seasonal // result[:,1]->trend //  result[:,2]->residual
            #series_d  = result[:,0]+result[:,2]
            ##series_d  = result[:,0]+result[:,1]+result[:,2]#check
#############################################################################
            series_d=temp1
            series_data = series_d[:-horizon]
            series_length = series_data.size
            n_val = int(np.round(series_length*.2))
            
            train = series_data[:-n_val]
            test = series_d[-(horizon+window_size):]
            #val = series_data[-(n_val+window_size+3):]
            val = series_data[-(2*n_val+window_size):]

            train_sequence = make_input(train, window_size,horizon)
            val_sequence = make_input(val,window_size,horizon)
            test_sequence = nonov_make_input(test,window_size,horizon)

            x_train = train_sequence[0]
            y_train =train_sequence[1]
            x_val = val_sequence[0]
            y_val = val_sequence[1]
            x_test = test_sequence[0]
            y_test = test_sequence[1]

############################################################################################
###############creating train, validation and test sets from Predictions####################
############################################################################################
            #current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
            #current_pred[np.argwhere(current_pred<=0)]=0.5
            #current_pred = np.log(current_pred)
####################################  Decomposition ##########################
            #result_pred = stl(ts(current_pred,frequency=frequency),"periodic")
            #result_pred=pandas2ri.ri2py(result_pred.rx2('time.series'))
            ##series_pred = result_pred[:,0]+result_pred[:,2]
            #series_p = result_pred[:,0]+result_pred[:,1]+result_pred[:,2]#check
#############################################################################
            ##series_pred=current_pred
            #series_pred=series_p[:-horizon]
            #train_p = series_pred[:-n_val]
            ##val_p = series_pred[-(n_val+window_size+3):]
            #val_p = series_pred[-(2*n_val+window_size):]
            #test_p = series_p[-(horizon+window_size):]

            #train_pred = make_k_input(train_p,window_size,horizon)
            #val_pred = make_k_input(val_p,window_size,horizon)
            #test_pred = nonov_make_k_input(test_p,window_size,horizon)


            ##train_pred=train_pred.reshape(train_pred.shape[0],horizon)
            ##val_pred=val_pred.reshape(val_pred.shape[0],horizon)
            ##test_pred=test_pred.reshape(test_pred.shape[0],horizon)
############################################################################################
########################    Creating inputs for Neural Network      ########################
############################################################################################
            #train_input = np.append(x_train,train_pred,axis=1)
            train_input = x_train
            #val_input = np.append(x_val,val_pred,axis=1)
            val_input = x_val
            #test_input = np.append(x_test,test_pred,axis=1)
            test_input = x_test

############################################################################################
########################       Neural Network Configuration         ########################
############################################################################################
            #input_data= Input(batch_shape=(None,window_size+horizon,1),name='input_data')
            input_data= Input(batch_shape=(None,window_size,1),name='input_data')

            branch_0 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
            branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
            branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
            branch_3=Flatten()(branch_2)
            net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

            model=Model(inputs=[input_data],outputs=net)
            callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF1.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

            model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)
            model.load_weights('MyModelCheckpoint/CIF1.h5')
            pred=model.predict({'input_data':test_input})
            pred=pred.reshape(pred.size)[:horizon]

            final_predictions[y,:horizon] = pred.reshape(horizon)
            smape1_pred+=sMAPE1(pred,test[-horizon:])
            smape_pred+=sMAPE(pred,test[-horizon:],horizon)
            #smape1_theta+=sMAPE1(np.exp(current_pred[-horizon:]),test[-horizon:])
            #smape_theta+=sMAPE(np.exp(current_pred[-horizon:]),test[-horizon:],horizon)
            pbar.update(1)
        np.savetxt('MyModelCheckpoint/prediction1.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(smape1_pred)
print(smape_pred)
print(datetime.datetime.now())

2019-07-22 01:07:50.161738


100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [09:38<00:00, 11.88s/it]


30.498979047852988
76.11706880400571
2019-07-22 01:17:28.850272


# Case 2:
##### Using Decomposition Method
##### Using Combination of Data and Prediction
##### Using 3Layers Conv (64) Neural Network 

In [9]:
print(datetime.datetime.now())
smape_pred=0
smape1_pred=0
smape_theta=0
smape1_theta=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    for horizon in [1]:
        final_predictions = np.zeros([data_length,12])
        for y in range(data_length):
############################################################################################
######################creating train, validation and test sets from Data####################
############################################################################################
            horizon = data.iloc[y].values[1]
            window_size=7
            current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
            rr = current_row.size
            rr = int(np.floor(rr*.25))
            temp1=current_row[rr:] # values after (rr)th position in current_row
            epsilon = 0.05     #check
            temp1[temp1<epsilon] = temp1[temp1<epsilon]+0.05
            temp1 = np.log(temp1)

####################################  Decomposition ##########################
            frequency = 7     #check
            result = stl(ts(temp1,frequency=frequency),"periodic")
            result=pandas2ri.ri2py(result.rx2('time.series')) # result[:,0]->seasonal // result[:,1]->trend //  result[:,2]->residual
            #series  = result[:,0]+result[:,2]
            series_d  = result[:,0]+result[:,1]+result[:,2]#check
#############################################################################
            #series=temp1
            series_data = series_d[:-horizon]
            series_length = series_data.size
            n_val = int(np.round(series_length*.2))
            #if n_val < horizon:
            #    n_val = horizon
            train = series_data[:-n_val]
            test = series_d[-(horizon+window_size):]
            #val = series_data[-(n_val+window_size+3):]
            val = series_data[-(2*n_val+window_size):]

            train_sequence = make_input(train, window_size,horizon)
            val_sequence = make_input(val,window_size,horizon)
            test_sequence = nonov_make_input(test,window_size,horizon)

            x_train = train_sequence[0]
            y_train =train_sequence[1]
            x_val = val_sequence[0]
            y_val = val_sequence[1]
            x_test = test_sequence[0]
            y_test = test_sequence[1]

############################################################################################
###############creating train, validation and test sets from Predictions####################
############################################################################################
            current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
            current_pred[np.argwhere(current_pred<=0)]=0.5
            current_pred = np.log(current_pred)
####################################  Decomposition ##########################
            result_pred = stl(ts(current_pred,frequency=frequency),"periodic")
            result_pred=pandas2ri.ri2py(result_pred.rx2('time.series'))
            #series_pred = result_pred[:,0]+result_pred[:,2]
            series_p = result_pred[:,0]+result_pred[:,1]+result_pred[:,2]#check
#############################################################################
            #series_pred=current_pred
            series_pred=series_p[:-horizon]
            train_p = series_pred[:-n_val]
            #val_p = series_pred[-(n_val+window_size+3):]
            val_p = series_pred[-(2*n_val+window_size):]
            test_p = series_p[-(horizon+window_size):]

            train_pred = make_k_input(train_p,window_size,horizon)
            val_pred = make_k_input(val_p,window_size,horizon)
            test_pred = nonov_make_k_input(test_p,window_size,horizon)


            #train_pred=train_pred.reshape(train_pred.shape[0],horizon)
            #val_pred=val_pred.reshape(val_pred.shape[0],horizon)
            #test_pred=test_pred.reshape(test_pred.shape[0],horizon)
############################################################################################
########################    Creating inputs for Neural Network      ########################
############################################################################################
            train_input = np.append(x_train,train_pred,axis=1)
            #train_input = x_train
            val_input = np.append(x_val,val_pred,axis=1)
            #val_input = x_val
            test_input = np.append(x_test,test_pred,axis=1)
            #test_input = x_test

############################################################################################
########################       Neural Network Configuration         ########################
############################################################################################
            input_data= Input(batch_shape=(None,window_size+horizon,1),name='input_data')

            branch_0 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
            branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
            branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
            branch_3=Flatten()(branch_2)
            net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

            model=Model(inputs=[input_data],outputs=net)
            callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF2.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

            model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)
            model.load_weights('MyModelCheckpoint/CIF2.h5')
            pred=model.predict({'input_data':test_input})
            pred=pred.reshape(pred.size)[:horizon]

            final_predictions[y,:horizon] = pred.reshape(horizon)
            smape1_pred+=sMAPE1(pred,test[-horizon:])
            smape_pred+=sMAPE(pred,test[-horizon:],horizon)
            smape1_theta+=sMAPE1(np.exp(current_pred[-horizon:]),test[-horizon:])
            smape_theta+=sMAPE(np.exp(current_pred[-horizon:]),test[-horizon:],horizon)
            pbar.update(1)
        np.savetxt('MyModelCheckpoint/prediction2.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print(smape1_pred)
print(smape_pred)
print(smape1_theta)
print(smape_theta)

2019-07-22 01:17:29.647274


 75%|█████████████████████████████████████████████████████████████▌                    | 54/72 [14:26<05:08, 17.16s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:434: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [19:57<00:00, 20.01s/it]


2019-07-22 01:37:27.270882
33.81067463491623
73.7802316577959
14.837416190789224
140.62648529102384


In [303]:
print(smape1_pred)
print(smape_pred)
print(smape1_theta)
print(smape_theta)


34.53051716362836
-----------------
73.95666938993689
-----------------
14.837416190789224
-----------------
140.62648529102384


# Case 3 :
##### Using Raw Data and Prediction (without Decomposition)
##### Using Combination of Data and Prediction
##### Using 3Layers Conv (64) Neural Network 

In [10]:
print(datetime.datetime.now())
smape_pred=0
smape1_pred=0
smape_theta=0
smape1_theta=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    for horizon in [1]:
        final_predictions = np.zeros([data_length,12])
        for y in range(data_length):
############################################################################################
######################creating train, validation and test sets from Data####################
############################################################################################
            horizon = data.iloc[y].values[1]
            window_size=7
            current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
            rr = current_row.size
            rr = int(np.floor(rr*.25))
            temp1=current_row[rr:] # values after (rr)th position in current_row
            epsilon = 0.05     #check
            temp1[temp1<epsilon] = temp1[temp1<epsilon]+0.05
            temp1 = np.log(temp1)

####################################  Decomposition ##########################
            #frequency = 7     #check
            #result = stl(ts(temp1,frequency=frequency),"periodic")
            #result=pandas2ri.ri2py(result.rx2('time.series')) # result[:,0]->seasonal // result[:,1]->trend //  result[:,2]->residual
            #series_d  = result[:,0]+result[:,2]
            #series_d  = result[:,0]+result[:,1]+result[:,2]#check
#############################################################################
            series_d=temp1
            series_data = series_d[:-horizon]
            series_length = series_data.size
            n_val = int(np.round(series_length*.2))
            
            train = series_data[:-n_val]
            test = series_d[-(horizon+window_size):]
            #val = series_data[-(n_val+window_size+3):]
            val = series_data[-(2*n_val+window_size):]

            train_sequence = make_input(train, window_size,horizon)
            val_sequence = make_input(val,window_size,horizon)
            test_sequence = nonov_make_input(test,window_size,horizon)

            x_train = train_sequence[0]
            y_train =train_sequence[1]
            x_val = val_sequence[0]
            y_val = val_sequence[1]
            x_test = test_sequence[0]
            y_test = test_sequence[1]

############################################################################################
###############creating train, validation and test sets from Predictions####################
############################################################################################
            current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
            current_pred[np.argwhere(current_pred<=0)]=0.5
            current_pred = np.log(current_pred)
####################################  Decomposition ##########################
            #result_pred = stl(ts(current_pred,frequency=frequency),"periodic")
            #result_pred=pandas2ri.ri2py(result_pred.rx2('time.series'))
            #series_p = result_pred[:,0]+result_pred[:,2]
            #series_p = result_pred[:,0]+result_pred[:,1]+result_pred[:,2]#check
#############################################################################
            series_p=current_pred
            series_pred=series_p[:-horizon]
            train_p = series_pred[:-n_val]
            #val_p = series_pred[-(n_val+window_size+3):]
            val_p = series_pred[-(2*n_val+window_size):]
            test_p = series_p[-(horizon+window_size):]

            train_pred = make_k_input(train_p,window_size,horizon)
            val_pred = make_k_input(val_p,window_size,horizon)
            test_pred = nonov_make_k_input(test_p,window_size,horizon)


            #train_pred=train_pred.reshape(train_pred.shape[0],horizon)
            #val_pred=val_pred.reshape(val_pred.shape[0],horizon)
            #test_pred=test_pred.reshape(test_pred.shape[0],horizon)
############################################################################################
########################    Creating inputs for Neural Network      ########################
############################################################################################
            train_input = np.append(x_train,train_pred,axis=1)
            #train_input = x_train
            val_input = np.append(x_val,val_pred,axis=1)
            #val_input = x_val
            test_input = np.append(x_test,test_pred,axis=1)
            #test_input = x_test

############################################################################################
########################       Neural Network Configuration         ########################
############################################################################################
            input_data= Input(batch_shape=(None,window_size+horizon,1),name='input_data')
            input_pred=Input(batch_shape=(None,horizon),name='input_pred')

            branch_0 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
            branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
            branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
            branch_3=Flatten()(branch_2)
            net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

            model=Model(inputs=[input_data],outputs=net)
            callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF3.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

            model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)
            model.load_weights('MyModelCheckpoint/CIF3.h5')
            pred=model.predict({'input_data':test_input})
            pred=pred.reshape(pred.size)[:horizon]

            final_predictions[y,:horizon] = pred.reshape(horizon)
            smape1_pred+=sMAPE1(pred,test[-horizon:])
            smape_pred+=sMAPE(pred,test[-horizon:],horizon)
            smape1_theta+=sMAPE1(np.exp(current_pred[-horizon:]),test[-horizon:])
            smape_theta+=sMAPE(np.exp(current_pred[-horizon:]),test[-horizon:],horizon)
            pbar.update(1)
        np.savetxt('MyModelCheckpoint/prediction3.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print(smape1_pred)
print(smape_pred)
print(smape1_theta)
print(smape_theta)

2019-07-22 01:37:28.020915


 75%|█████████████████████████████████████████████████████████████▌                    | 54/72 [27:15<09:02, 30.15s/it]C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:434: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [36:39<00:00, 32.95s/it]


2019-07-22 02:14:07.145303
33.43674357107893
74.0308812520432
14.837416190789224
140.62648529102384


In [305]:
print(smape1_pred)
print('-----------------')
print(smape_pred)
print('-----------------')
print(smape1_theta)
print('-----------------')
print(smape_theta)


33.91056424227694
-----------------
74.02260568245515
-----------------
14.837416190789224
-----------------
140.62648529102384


# Case 4 :
##### Using Data to feed into Neural Network
##### Using autoencoder on Prediction to achive the size 32
##### Merging Data and Prediction
##### Using 3Layers Conv (64) Neural Network + autoencoder

In [11]:
print(datetime.datetime.now())
smape_pred=0
smape1_pred=0
smape_theta=0
smape1_theta=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    for horizon in [1]:
        final_predictions = np.zeros([data_length,12])
        for y in range(data_length):
############################################################################################
######################creating train, validation and test sets from Data####################
############################################################################################
            horizon = data.iloc[y].values[1]
            window_size=7
            current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
            temp1=current_row
            epsilon = 0.05     #check
            temp1[temp1<epsilon] = temp1[temp1<epsilon]+0.05
            temp1 = np.log(temp1)

            ########################  Decomposition ##########################
            frequency = 7     #check
            result = stl(ts(temp1,frequency=frequency),"periodic")
            result=pandas2ri.ri2py(result.rx2('time.series')) # result[:,0]->seasonal // result[:,1]->trend //  result[:,2]->residual
            series_d  = result[:,0]+result[:,2]
            #################################################################
            #series=temp1
            series_data = series_d[:-horizon]
            series_length = series_data.size
            n_val = int(np.round(series_length*.2))

            train = series_data[:-n_val]
            test = series_d[-(horizon+window_size):]
            val = series_data[-(2*n_val+window_size):]

            train_sequence = make_input(train, window_size,horizon)
            val_sequence = make_input(val,window_size,horizon)
            test_sequence = nonov_make_input(test,window_size,horizon)

            x_train = train_sequence[0]
            y_train =train_sequence[1]
            x_val = val_sequence[0]
            y_val = val_sequence[1]
            x_test = test_sequence[0]
            y_test = test_sequence[1]

################################################################################
###creating train, validation and test sets from Predictions####################
################################################################################
            current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
            current_pred[np.argwhere(current_pred<=0)]=0.5
            current_pred = np.log(current_pred)
            ########################  Decomposition ##########################
            result_pred = stl(ts(current_pred,frequency=frequency),"periodic")
            result_pred=pandas2ri.ri2py(result_pred.rx2('time.series'))
            series_p = result_pred[:,0]+result_pred[:,2]
            #################################################################

            #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^#
            #^^^^^^^^^^^^^^^^^^^^^^^^^  Autoencoder  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^#
            series_p_auto=series_p.reshape(1,series_p.shape[0])
            series_p_auto=pd.DataFrame(series_p_auto)
            ncol = series_p_auto.size
            n_val_p=int(np.round(ncol*.2))
            encoding_dim = ncol-horizon-n_val_p
            input_dim = Input(shape = (ncol, ))
            # Encoder Layers
            encoded1 = Dense(ncol-horizon, activation = 'relu')(input_dim)
            encoded2 = Dense(encoding_dim, activation = 'relu')(encoded1)
            # Decoder Layers
            decoded1 = Dense(ncol-horizon, activation = 'relu')(encoded2)
            decoded2 = Dense(ncol, activation = 'sigmoid')(decoded1)
            # Combine Encoder and Deocder layers
            autoencoder = Model(inputs = input_dim, outputs = decoded2)
            autoencoder.compile(optimizer = 'adadelta', loss = 'mean_squared_logarithmic_error')
            autoencoder.fit(series_p_auto, series_p_auto, nb_epoch = 10, batch_size = 8, shuffle = False) 
            encoder = Model(inputs = input_dim, outputs = encoded2)
            encoded_input = Input(shape = (encoding_dim, ))
            series_p_auto2 = np.array(encoder.predict(series_p_auto))
            series_p_auto2=series_p_auto2.reshape(series_p_auto2.shape[1],)

            #############^^^^^^^^^^^^&&&&&&&&&&&&&&&&&&^^^^^^^^^^^^#############
            series_p_auto2=pd.DataFrame(series_p_auto2).transpose()
            ncol2 = series_p_auto2.size
            n_val_p2=int(np.round(ncol2*.2))
            encoding_dim2 = 32
            input_dim2 = Input(shape = (ncol2, ))
            middle_dim=ncol2-horizon
            if ncol2< 32:
                middle_dim=ncol2+horizon
    
            # Encoder Layers
            encoded12 = Dense(middle_dim, activation = 'relu')(input_dim2)
            encoded22 = Dense(encoding_dim2, activation = 'relu')(encoded12)
            # Decoder Layers
            decoded12 = Dense(middle_dim, activation = 'relu')(encoded22)
            decoded22 = Dense(ncol2, activation = 'sigmoid')(decoded12)
            # Combine Encoder and Deocder layers
            autoencoder = Model(inputs = input_dim2, outputs = decoded22)
            autoencoder.compile(optimizer = 'adadelta', loss = 'mean_squared_logarithmic_error')
            autoencoder.fit(series_p_auto2, series_p_auto2, nb_epoch = 10, batch_size = 8, shuffle = False) 
            encoder2 = Model(inputs = input_dim2, outputs = encoded22)
            encoded_input2 = Input(shape = (encoding_dim2, ))
            series_p_auto3 = np.array(encoder2.predict(series_p_auto2))
            series_p_auto3=series_p_auto3.reshape(series_p_auto3.shape[1],)
            #############^^^^^^^^^^^^&&&&&&&&&&&&&&&&&&^^^^^^^^^^^^#############

            #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^#
            #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^#

############################################################################################
########################    Creating inputs for Neural Network      ########################
############################################################################################
            train_input = x_train
            val_input = x_val
            test_input = x_test

############################################################################################
########################       Neural Network Configuration         ########################
############################################################################################
            series_p_auto4=np.reshape(series_p_auto3, (1,32))
            predict_tensor=tf.convert_to_tensor(series_p_auto4)
    
            input_data= Input(batch_shape=(None,window_size+horizon,1),name='input_data')

            branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
            branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
            branch_2=add([branch_1,predict_tensor])
            branch_3 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_2)
            branch_4=Flatten()(branch_3)
            net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_4)


            model=Model(inputs=[input_data],outputs=net)
            callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF4.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))
            model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)
            model.load_weights('MyModelCheckpoint/CIF4.h5')
            pred=model.predict({'input_data':test_input})
            pred=pred.reshape(pred.size)[:horizon]

            final_predictions[y,:horizon] = pred.reshape(horizon)
            smape1_pred+=sMAPE1(pred,test[-horizon:])
            smape_pred+=sMAPE(pred,test[-horizon:],horizon)
            smape1_theta+=sMAPE1(np.exp(current_pred[-horizon:]),test[-horizon:])
            smape_theta+=sMAPE(np.exp(current_pred[-horizon:]),test[-horizon:],horizon)
            pbar.update(1)
        np.savetxt('MyModelCheckpoint/prediction4.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print(smape1_pred)
print(smape_pred)
print(smape1_theta)
print(smape_theta)

2019-07-22 02:14:08.708078


  0%|                                                                                           | 0/72 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:83: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/10
1/1 [==============================] - 11s 11s/step - loss: 0.1622
Epoch 2/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1619
Epoch 3/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1617
Epoch 4/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1615
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1613
Epoch 6/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1611
Epoch 7/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1609
Epoch 8/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1607
Epoch 9/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1605
Epoch 10/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1603


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:108: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/10
1/1 [==============================] - 11s 11s/step - loss: 0.1611
Epoch 2/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1609
Epoch 3/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1607
Epoch 4/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1605
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1602
Epoch 6/10
1/1 [==============================] - 0s 0us/step - loss: 0.1600
Epoch 7/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1598
Epoch 8/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1596
Epoch 9/10
1/1 [==============================] - 0s 0us/step - loss: 0.1594
Epoch 10/10
1/1 [==============================] - 0s 0us/step - loss: 0.1591


...

Epoch 1/10
1/1 [==============================] - 28s 28s/step - loss: 0.1494
Epoch 2/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1492
Epoch 3/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1489
Epoch 4/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1486
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1483
Epoch 6/10
1/1 [==============================] - 0s 31ms/step - loss: 0.1480
Epoch 7/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1477
Epoch 8/10
1/1 [==============================] - 0s 31ms/step - loss: 0.1475
Epoch 9/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1472
Epoch 10/10
1/1 [==============================] - 0s 31ms/step - loss: 0.1470
Epoch 1/10
1/1 [==============================] - 28s 28s/step - loss: 0.1489
Epoch 2/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1486
Epoch 3/10
1/1 [==============================] - 0s 31ms/step 

 99%|█████████████████████████████████████████████████████████████████████████████▉ | 71/72 [2:07:43<02:29, 149.57s/it]

Epoch 1/10
1/1 [==============================] - 26s 26s/step - loss: 0.1470
Epoch 2/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1467
Epoch 3/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1464
Epoch 4/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1461
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1458
Epoch 6/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1455
Epoch 7/10
1/1 [==============================] - 0s 31ms/step - loss: 0.1452
Epoch 8/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1448
Epoch 9/10
1/1 [==============================] - 0s 31ms/step - loss: 0.1445
Epoch 10/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1441
Epoch 1/10
1/1 [==============================] - 27s 27s/step - loss: 0.1468
Epoch 2/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1466
Epoch 3/10
1/1 [==============================] - 0s 16ms/step 

100%|███████████████████████████████████████████████████████████████████████████████| 72/72 [2:10:15<00:00, 150.41s/it]


2019-07-22 04:24:24.510996
18.181252942504976
117.4439649408635
14.508896403502735
143.93485641624216


# Case 5 :
##### using Raw Data (no Decomposition)
##### Using Data to feed into Neural Network
##### Using autoencoder on Prediction to achive the size 32
##### Merging Data and Prediction
##### Using 3Layers Conv (64) Neural Network + autoencoder

In [7]:
print(datetime.datetime.now())
smape_pred=0
smape1_pred=0
smape_theta=0
smape1_theta=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    for horizon in [1]:
        final_predictions = np.zeros([data_length,12])
        for y in range(data_length):
############################################################################################
######################creating train, validation and test sets from Data####################
############################################################################################
            horizon = data.iloc[y].values[1]
            window_size=7
            current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
            temp1=current_row
            epsilon = 0.05     #check
            temp1[temp1<epsilon] = temp1[temp1<epsilon]+0.05
            temp1 = np.log(temp1)

            ########################  Decomposition ##########################
            #frequency = 7     #check
            #result = stl(ts(temp1,frequency=frequency),"periodic")
            #result=pandas2ri.ri2py(result.rx2('time.series')) # result[:,0]->seasonal // result[:,1]->trend //  result[:,2]->residual
            #series_d  = result[:,0]+result[:,2]
            #################################################################
            series_d=temp1
            series_data = series_d[:-horizon]
            series_length = series_data.size
            n_val = int(np.round(series_length*.2))

            train = series_data[:-n_val]
            test = series_d[-(horizon+window_size):]
            val = series_data[-(2*n_val+window_size):]

            train_sequence = make_input(train, window_size,horizon)
            val_sequence = make_input(val,window_size,horizon)
            test_sequence = nonov_make_input(test,window_size,horizon)

            x_train = train_sequence[0]
            y_train =train_sequence[1]
            x_val = val_sequence[0]
            y_val = val_sequence[1]
            x_test = test_sequence[0]
            y_test = test_sequence[1]

################################################################################
###creating train, validation and test sets from Predictions####################
################################################################################
            current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
            current_pred[np.argwhere(current_pred<=0)]=0.5
            current_pred = np.log(current_pred)
            ########################  Decomposition ##########################
            #result_pred = stl(ts(current_pred,frequency=frequency),"periodic")
            #result_pred=pandas2ri.ri2py(result_pred.rx2('time.series'))
            #series_p = result_pred[:,0]+result_pred[:,2]
            series_p=current_pred
            #################################################################

            #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^#
            #^^^^^^^^^^^^^^^^^^^^^^^^^  Autoencoder  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^#
            series_p_auto=series_p.reshape(1,series_p.shape[0])
            series_p_auto=pd.DataFrame(series_p_auto)
            ncol = series_p_auto.size
            n_val_p=int(np.round(ncol*.2))
            encoding_dim = ncol-horizon-n_val_p
            input_dim = Input(shape = (ncol, ))
            # Encoder Layers
            encoded1 = Dense(ncol-horizon, activation = 'relu')(input_dim)
            encoded2 = Dense(encoding_dim, activation = 'relu')(encoded1)
            # Decoder Layers
            decoded1 = Dense(ncol-horizon, activation = 'relu')(encoded2)
            decoded2 = Dense(ncol, activation = 'sigmoid')(decoded1)
            # Combine Encoder and Deocder layers
            autoencoder = Model(inputs = input_dim, outputs = decoded2)
            autoencoder.compile(optimizer = 'adadelta', loss = 'mean_squared_logarithmic_error')
            autoencoder.fit(series_p_auto, series_p_auto, nb_epoch = 10, batch_size = 8, shuffle = False) 
            encoder = Model(inputs = input_dim, outputs = encoded2)
            encoded_input = Input(shape = (encoding_dim, ))
            series_p_auto2 = np.array(encoder.predict(series_p_auto))
            series_p_auto2=series_p_auto2.reshape(series_p_auto2.shape[1],)

            #############^^^^^^^^^^^^&&&&&&&&&&&&&&&&&&^^^^^^^^^^^^#############
            series_p_auto2=pd.DataFrame(series_p_auto2).transpose()
            ncol2 = series_p_auto2.size
            n_val_p2=int(np.round(ncol2*.2))
            encoding_dim2 = 32
            input_dim2 = Input(shape = (ncol2, ))
            middle_dim=ncol2-horizon
            if ncol2< 32:
                middle_dim=ncol2+horizon
    
            # Encoder Layers
            encoded12 = Dense(middle_dim, activation = 'relu')(input_dim2)
            encoded22 = Dense(encoding_dim2, activation = 'relu')(encoded12)
            # Decoder Layers
            decoded12 = Dense(middle_dim, activation = 'relu')(encoded22)
            decoded22 = Dense(ncol2, activation = 'sigmoid')(decoded12)
            # Combine Encoder and Deocder layers
            autoencoder = Model(inputs = input_dim2, outputs = decoded22)
            autoencoder.compile(optimizer = 'adadelta', loss = 'mean_squared_logarithmic_error')
            autoencoder.fit(series_p_auto2, series_p_auto2, nb_epoch = 10, batch_size = 8, shuffle = False) 
            encoder2 = Model(inputs = input_dim2, outputs = encoded22)
            encoded_input2 = Input(shape = (encoding_dim2, ))
            series_p_auto3 = np.array(encoder2.predict(series_p_auto2))
            series_p_auto3=series_p_auto3.reshape(series_p_auto3.shape[1],)
            #############^^^^^^^^^^^^&&&&&&&&&&&&&&&&&&^^^^^^^^^^^^#############

            #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^#
            #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^#

############################################################################################
########################    Creating inputs for Neural Network      ########################
############################################################################################
            train_input = x_train
            val_input = x_val
            test_input = x_test

############################################################################################
########################       Neural Network Configuration         ########################
############################################################################################
            series_p_auto4=np.reshape(series_p_auto3, (1,32))
            predict_tensor=tf.convert_to_tensor(series_p_auto4)
    
            input_data= Input(batch_shape=(None,window_size+horizon,1),name='input_data')

            branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
            branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
            branch_2=add([branch_1,predict_tensor])
            branch_3 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_2)
            branch_4=Flatten()(branch_3)
            net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_4)


            model=Model(inputs=[input_data],outputs=net)
            callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF5.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))
            model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)
            model.load_weights('MyModelCheckpoint/CIF5.h5')
            pred=model.predict({'input_data':test_input})
            pred=pred.reshape(pred.size)[:horizon]

            final_predictions[y,:horizon] = pred.reshape(horizon)
            smape1_pred+=sMAPE1(pred,test[-horizon:])
            smape_pred+=sMAPE(pred,test[-horizon:],horizon)
            smape1_theta+=sMAPE1(np.exp(current_pred[-horizon:]),test[-horizon:])
            smape_theta+=sMAPE(np.exp(current_pred[-horizon:]),test[-horizon:],horizon)
            pbar.update(1)
        np.savetxt('MyModelCheckpoint/prediction5.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print(smape1_pred)
print(smape_pred)
print(smape1_theta)
print(smape_theta)

2019-07-22 13:15:40.403288


  0%|                                                                                           | 0/72 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:83: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/10
1/1 [==============================] - 0s 251ms/step - loss: 2.9469
Epoch 2/10
1/1 [==============================] - 0s 8ms/step - loss: 2.8660
Epoch 3/10
1/1 [==============================] - 0s 4ms/step - loss: 2.8018
Epoch 4/10
1/1 [==============================] - 0s 4ms/step - loss: 2.7414
Epoch 5/10
1/1 [==============================] - 0s 4ms/step - loss: 2.6951
Epoch 6/10
1/1 [==============================] - 0s 4ms/step - loss: 2.6316
Epoch 7/10
1/1 [==============================] - 0s 4ms/step - loss: 2.5669
Epoch 8/10
1/1 [==============================] - 0s 4ms/step - loss: 2.5013
Epoch 9/10
1/1 [==============================] - 0s 0us/step - loss: 2.4281
Epoch 10/10
1/1 [==============================] - 0s 0us/step - loss: 2.3567


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:108: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/10
1/1 [==============================] - 0s 285ms/step - loss: 0.9437
Epoch 2/10
1/1 [==============================] - 0s 5ms/step - loss: 0.9196
Epoch 3/10
1/1 [==============================] - 0s 0us/step - loss: 0.8982
Epoch 4/10
1/1 [==============================] - 0s 0us/step - loss: 0.8787
Epoch 5/10
1/1 [==============================] - 0s 4ms/step - loss: 0.8607
Epoch 6/10
1/1 [==============================] - 0s 0us/step - loss: 0.8438
Epoch 7/10
1/1 [==============================] - 0s 4ms/step - loss: 0.8278
Epoch 8/10
1/1 [==============================] - 0s 4ms/step - loss: 0.8129
Epoch 9/10
1/1 [==============================] - 0s 0us/step - loss: 0.7989
Epoch 10/10
1/1 [==============================] - 0s 0us/step - loss: 0.7850


...

Epoch 1/10
1/1 [==============================] - 10s 10s/step - loss: 6.0486
Epoch 2/10
1/1 [==============================] - 0s 16ms/step - loss: 5.9459
Epoch 3/10
1/1 [==============================] - 0s 0us/step - loss: 5.8883
Epoch 4/10
1/1 [==============================] - 0s 16ms/step - loss: 5.8412
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 5.7934
Epoch 6/10
1/1 [==============================] - 0s 0us/step - loss: 5.7471
Epoch 7/10
1/1 [==============================] - 0s 16ms/step - loss: 5.6963
Epoch 8/10
1/1 [==============================] - 0s 16ms/step - loss: 5.6470
Epoch 9/10
1/1 [==============================] - 0s 16ms/step - loss: 5.6031
Epoch 10/10
1/1 [==============================] - 0s 0us/step - loss: 5.5632
Epoch 1/10
1/1 [==============================] - 10s 10s/step - loss: 1.4626
Epoch 2/10
1/1 [==============================] - 0s 16ms/step - loss: 1.4302
Epoch 3/10
1/1 [==============================] - 0s 16ms/step - l

 99%|████████████████████████████████████████████████████████████████████████████████▊ | 71/72 [38:57<01:03, 63.47s/it]

Epoch 1/10
1/1 [==============================] - 10s 10s/step - loss: 6.4539
Epoch 2/10
1/1 [==============================] - 0s 16ms/step - loss: 6.3166
Epoch 3/10
1/1 [==============================] - 0s 16ms/step - loss: 6.2289
Epoch 4/10
1/1 [==============================] - 0s 0us/step - loss: 6.1279
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 6.0485
Epoch 6/10
1/1 [==============================] - 0s 16ms/step - loss: 5.9615
Epoch 7/10
1/1 [==============================] - 0s 0us/step - loss: 5.8815
Epoch 8/10
1/1 [==============================] - 0s 16ms/step - loss: 5.7976
Epoch 9/10
1/1 [==============================] - 0s 16ms/step - loss: 5.7375
Epoch 10/10
1/1 [==============================] - 0s 0us/step - loss: 5.6370
Epoch 1/10
1/1 [==============================] - 10s 10s/step - loss: 1.2742
Epoch 2/10
1/1 [==============================] - 0s 16ms/step - loss: 1.2503
Epoch 3/10
1/1 [==============================] - 0s 0us/step - lo

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [40:05<00:00, 64.68s/it]


2019-07-22 13:55:45.673397
30.471826972210287
75.73488431304342
14.837416190789224
140.62648529102384
